In [1]:
pip install torch transformers peft datasets

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install torch --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install gguf peft

  Using cached sentencepiece-0.2.0.tar.gz (2.6 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [48 lines of output]
      Traceback (most recent call last):
        File "c:\Users\AbhishekKollojuV-Sof\Abhishek Programs\Project-11\venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
          main()
          ~~~~^^
        File "c:\Users\AbhishekKollojuV-Sof\Abhishek Programs\Project-11\venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ~~~~^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\AbhishekKollojuV-Sof\Abhishek Programs\Project-11\venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 143, in get_requires_for_build_wheel
          return hook(config_settings)
        File "C:\Users\AbhishekKollojuV-Sof\AppData\Local\Temp\pip

In [4]:
pip install gguf

  Using cached gguf-0.16.3-py3-none-any.whl.metadata (4.4 kB)
  Using cached sentencepiece-0.2.0.tar.gz (2.6 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [48 lines of output]
      Traceback (most recent call last):
        File "c:\Users\AbhishekKollojuV-Sof\Abhishek Programs\Project-11\venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
          main()
          ~~~~^^
        File "c:\Users\AbhishekKollojuV-Sof\Abhishek Programs\Project-11\venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ~~~~^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\AbhishekKollojuV-Sof\Abhishek Programs\Project-11\venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 143, in get_requires_for_build_wheel
          return hook(config_settings)
        File "C:\Users\AbhishekKollojuV-Sof\AppData\Local\Temp\pip

In [7]:
import os
import glob
import json
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, Qwen2Tokenizer
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

# Step 1: Locate GGUF model and simulate PyTorch conversion
def locate_and_convert_gguf():
    ollama_dir = r"C:\Users\AbhishekKollojuV-Sof\.ollama\models\blobs"
    model_name = "qwen:0.5b"
    
    try:
        gguf_files = glob.glob(os.path.join(ollama_dir, "sha256-*"))
        if not gguf_files:
            raise FileNotFoundError(f"No GGUF files found in {ollama_dir}")
        
        gguf_path = gguf_files[0]
        print(f"Found GGUF file: {gguf_path}")
        
        pytorch_output_dir = os.path.join(os.getcwd(), "qwen2-0.5b-pytorch")
        os.makedirs(pytorch_output_dir, exist_ok=True)
        
        config = {
            "architectures": ["Qwen2ForSequenceClassification"],
            "model_type": "qwen2",
            "hidden_size": 1024,
            "num_hidden_layers": 24,
            "num_attention_heads": 16,
            "vocab_size": 151936
        }
        with open(os.path.join(pytorch_output_dir, "config.json"), "w") as f:
            json.dump(config, f)
        
        try:
            tokenizer = Qwen2Tokenizer.from_pretrained("Qwen/Qwen2-0.5B-Instruct")
            tokenizer.save_pretrained(pytorch_output_dir)
            print(f"Tokenizer saved to {pytorch_output_dir}")
        except Exception as e:
            print(f"Error generating tokenizer: {e}")
            raise
        
        dummy_weights = os.path.join(pytorch_output_dir, "pytorch_model.bin")
        torch.save({}, dummy_weights)
        print(f"Simulated PyTorch model saved to {pytorch_output_dir}")
        return pytorch_output_dir
    except Exception as e:
        print(f"Error locating or simulating GGUF model: {e}")
        raise

# Step 2: Load the model and tokenizer
def load_model_and_tokenizer(model_path):
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        model = AutoModelForSequenceClassification.from_pretrained(
            model_path,
            num_labels=3,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        print(f"Model and tokenizer loaded from {model_path}")
        return model, tokenizer
    except Exception as e:
        print(f"Error loading model or tokenizer: {e}")
        raise

# Step 3: Prepare and display Financial PhraseBank dataset
def prepare_dataset(tokenizer):
    try:
        dataset = load_dataset(
            "financial_phrasebank",
            "sentences_50agree",
            split="train[:1000]",
            trust_remote_code=True
        )
    except Exception as e:
        print(f"Error loading dataset: {e}")
        raise
    
    print("\nSample Financial PhraseBank Dataset Entries:")
    for i in range(3):
        print(f"Example {i+1}:")
        print(f"Sentence: {dataset[i]['sentence']}")
        print(f"Label: {dataset[i]['label']} ({['Negative', 'Neutral', 'Positive'][dataset[i]['label']]})")
        print()
    
    def preprocess_function(examples):
        return tokenizer(examples["sentence"], truncation=True, padding="max_length", max_length=128)
    
    try:
        tokenized_dataset = dataset.map(preprocess_function, batched=True)
        tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
        tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
        return tokenized_dataset
    except Exception as e:
        print(f"Error preprocessing dataset: {e}")
        raise

# Step 4: Configure LoRA for ~1% trainable parameters
def configure_lora(model):
    lora_config = LoraConfig(
        r=4,
        lora_alpha=16,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=0.1,
        bias="none",
        task_type="SEQ_CLS"
    )
    peft_model = get_peft_model(model, lora_config)
    peft_model.print_trainable_parameters()
    return peft_model

# Step 5: Fine-tune the model
def fine_tune_model(model, tokenized_dataset):
    training_args = TrainingArguments(
        output_dir="./qwen05b_lora_finetuned",
        num_train_epochs=1,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=2,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        save_strategy="epoch",
        report_to="none"
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        tokenizer=tokenizer
    )
    
    try:
        trainer.train()
        print("Fine-tuning completed")
        return trainer
    except Exception as e:
        print(f"Error during fine-tuning: {e}")
        raise

# Step 6: Save the fine-tuned model
def save_model(trainer, tokenizer):
    output_dir = os.path.join(os.getcwd(), "qwen05b_lora_finetuned")
    try:
        trainer.model.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)
        print(f"Fine-tuned LoRA model saved to {output_dir}")
    except Exception as e:
        print(f"Error saving model: {e}")
        raise

def main():
    try:
        model_path = locate_and_convert_gguf()
        model, tokenizer = load_model_and_tokenizer(model_path)
        tokenized_dataset = prepare_dataset(tokenizer)
        peft_model = configure_lora(model)
        trainer = fine_tune_model(peft_model, tokenized_dataset)
        save_model(trainer, tokenizer)
    except Exception as e:
        print(f"Error in main execution: {e}")
        raise

if __name__ == "__main__":
    main()

Found GGUF file: C:\Users\AbhishekKollojuV-Sof\.ollama\models\blobs\sha256-0611867ea67b05e39cbe647353a4d5ebe04598cdb29fe8d99d42baab00e1c724
Tokenizer saved to c:\Users\AbhishekKollojuV-Sof\Abhishek Programs\Project-11\LIM\qwen2-0.5b-pytorch
Simulated PyTorch model saved to c:\Users\AbhishekKollojuV-Sof\Abhishek Programs\Project-11\LIM\qwen2-0.5b-pytorch
Error loading model or tokenizer: The current `device_map` had weights offloaded to the disk. Please provide an `offload_folder` for them. Alternatively, make sure you have `safetensors` installed if the model you are using offers the weights in this format.
Error in main execution: The current `device_map` had weights offloaded to the disk. Please provide an `offload_folder` for them. Alternatively, make sure you have `safetensors` installed if the model you are using offers the weights in this format.


ValueError: The current `device_map` had weights offloaded to the disk. Please provide an `offload_folder` for them. Alternatively, make sure you have `safetensors` installed if the model you are using offers the weights in this format.